# ING Lion's Den 2024


## Team: RiskBusters

 Michał Bryzik, Michał Niegierewicz, Kacper Gruca, Jan Ślusarek

Plan dzialania: (Jbc to kazdy podaje tu swoje propozycje, jak sie kto z czym np. niezgadza niech napisze w nawiasie)

1. Wczytac dane i szybko je opisac liczebnosc nulle itd. 
2. Ja bym tutaj posortowal zmienne w macierzy tzn. kolumna 1-16 numeryczne a od 17-do konca binarne
3. Wlasciwe czyszczenie danych, elminacja, aproksymacja, asocjacja itd.
4. Statystyki opisowe, wykresy (rozklady, heatmapy, boxploty itd. )
5. Transformacja danych MinMax, standaryzacja itd. (ogolnie manipulacja tak zeby w razie czego logit byl jak najbardziej dojebany)
6. Trenowanie Logita
7. Wybor najlepszego i sprawdzenie z zalozeniami 

In [952]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [953]:
data=pd.read_csv('../data/input/development_sample.csv') 
data.head() 

,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,Var6,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
ID,,,,,,,,,,,,,,,,,,,,,
36034977,32653719,03Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4800,15,1,...,8,8,0,4977.87,14880.28,0,0,70,3899,0.485674
36034978,32832365,04Feb2010 0:00:00,0.0,Approved,2,2.0,1.0,6800,18,6,...,1,1,29400,7922.90,6534.84,0,0,0,3899,0.267045
36034979,32544742,07Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4600,18,1,...,0,0,0,3174.15,16974.69,0,0,0,3899,0.793579
36034980,32395830,09Feb2010 0:00:00,NaN,Rejected,2,2.0,2.0,7900,48,1,...,2,2,0,4972.82,3484.36,0,0,10,3899,0.533040
36034981,32592943,13Feb2010 0:00:00,0.0,Approved,2,1.0,1.0,9900,63,3,...,3,3,0,2278.22,9630.53,0,0,10,3899,0.781269


In [954]:
data.shape 

(5000, 35)

In [955]:
data.isnull().sum() 

customer_id              0
application_date         0
target                1353
Application_status       0
Var1                     0
Var2                   139
Var3                   139
Var4                     0
Var5                     0
Var6                     0
Var7                     0
Var8                  2905
Var9                     0
Var10                 3719
Var11                    0
Var12                 3719
Var13                    0
Var14                    0
Var15                    0
Var16                    0
Var17                    0
Var18                 3721
Var19                 2905
Var20                    0
Var21                    0
Var22                    0
Var23                    0
Var24                    0
Var25                 1023
Var26                 1961
Var27                    0
Var28                    0
Var29                    0
Var30                    0
_r_                      0
dtype: int64

In [956]:
data.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 36034977 to 36039976
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         5000 non-null   int64  
 1   application_date    5000 non-null   object 
 2   target              3647 non-null   float64
 3   Application_status  5000 non-null   object 
 4   Var1                5000 non-null   int64  
 5   Var2                4861 non-null   float64
 6   Var3                4861 non-null   float64
 7   Var4                5000 non-null   int64  
 8   Var5                5000 non-null   int64  
 9   Var6                5000 non-null   int64  
 10  Var7                5000 non-null   float64
 11  Var8                2095 non-null   float64
 12  Var9                5000 non-null   int64  
 13  Var10               1281 non-null   float64
 14  Var11               5000 non-null   int64  
 15  Var12               1281 non-null   float64


## Clean dataset

### Define columns by datetypes and sort df

We define the target type of the column basing on the description file and exploring data

In [957]:
# Define columns
date_columns = ['application_date', 'Var13'] 
numeric_columns = ['ID', 'customer_id', '_r_', 'Var9', 'Var10']  
categorical_columns = []
binary_columns = ['target', 'Application_status'] 

In [958]:
# Checking the data type for the rest of columns 
for column in data.columns:
    if column in date_columns + numeric_columns + binary_columns:
        continue  
    elif column in ['Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']:
        numeric_columns.append(column)
    elif column in ['Var2', 'Var3', 'Var11', 'Var12', 'Var14']:
        categorical_columns.append(column)
    elif column in ['Var18', 'Var19', 'Var27', 'Var28']:
        binary_columns.append(column)

In [959]:
print(date_columns)
print(numeric_columns)
print(categorical_columns)
print(binary_columns)

['application_date', 'Var13']
['ID', 'customer_id', '_r_', 'Var9', 'Var10', 'Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']
['Var2', 'Var3', 'Var11', 'Var12', 'Var14']
['target', 'Application_status', 'Var18', 'Var19', 'Var27', 'Var28']


In [960]:
# Make a copy of main data, from this moment we will work on this dataset
df = data.copy()

In [961]:
# Assumed sequence for columns sorting 
sorted_columns = date_columns + numeric_columns + categorical_columns + binary_columns

# Choose of the sorted value by defined categories
df = data[sorted_columns]

KeyError: "['ID'] not in index"

In [950]:
df.head()
# Some of the records for Var14 are Nulls, it requires analysis

,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,Var6,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
ID,,,,,,,,,,,,,,,,,,,,,
36034977,32653719,03Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4800,15,1,...,8,8,0,4977.87,14880.28,0,0,70,3899,0.485674
36034978,32832365,04Feb2010 0:00:00,0.0,Approved,2,2.0,1.0,6800,18,6,...,1,1,29400,7922.90,6534.84,0,0,0,3899,0.267045
36034979,32544742,07Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4600,18,1,...,0,0,0,3174.15,16974.69,0,0,0,3899,0.793579
36034980,32395830,09Feb2010 0:00:00,NaN,Rejected,2,2.0,2.0,7900,48,1,...,2,2,0,4972.82,3484.36,0,0,10,3899,0.533040
36034981,32592943,13Feb2010 0:00:00,0.0,Approved,2,1.0,1.0,9900,63,3,...,3,3,0,2278.22,9630.53,0,0,10,3899,0.781269


### Handle date variables

In [859]:
# Print date columns
date_columns

['application_date', 'Var13']

#### Change the type of columns

In [860]:
# Change date columns to datetime type
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['Var13'] = pd.to_datetime(df['Var13'], format='%d%b%Y', errors='coerce')         # Var13 - irregular date format

In [861]:
# Check naN values after transformation
print(df['Var13'].isnull().sum(0))
print(df['application_date'].isnull().sum(0))

784
0


After transformation some records for Var13 are empty, we need to analyze it

In [862]:
df.head()

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,2010-02-01,2004-11-26,11034977,32537148,0.267045,5640,NaN,1,7800,99,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,2010-02-01,1996-09-16,11034978,32761663,0.533040,3000,NaN,1,11100,78,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,2010-02-01,1989-05-06,11034979,32701063,0.861555,6240,4300.0,2,2400,15,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,2010-02-01,1987-12-06,11034980,32386786,0.506451,4560,2500.0,3,11800,30,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,2010-02-02,1989-05-18,11034981,32692110,0.210913,5160,NaN,1,10200,72,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


#### Analyze Var13

In [863]:
# Check indexes where conversion caused 'NaT' values
nan_indices = df['Var13'].isna()

# Compare original values of 'Var13' which after conversion became 'NaT'
original_values_with_nan = data.loc[nan_indices, 'Var13']

original_values_with_nan

5        31Dec9999
56       31Dec9999
67       31Dec9999
86       31Dec9999
95       31Dec9999
           ...    
49510    31Dec9999
49537    31Dec9999
49617    31Dec9999
49635    31Dec9999
49765    31Dec9999
Name: Var13, Length: 784, dtype: object

In [864]:
# Check unique values of 'Var13' which after conversion became 'NaT'
original_values_with_nan.unique()

array(['31Dec9999'], dtype=object)

Summarizing we have 784 cases in Var13 with value '31Dec9999'

We treat them as data error and for this moment leave with NAs

#### Create working_months column

Var13 = employment date

working_moths = Var13 - application_date


We would like to calculate the number of working months before applying for credit, which will indicate the employee's duration at the given position from employment to loan application.

NAs from Var13 are replaced with 0

We also want to have this column first

In [865]:
# Create a new colum working_months
df['working_months'] = ((df['application_date'] - df['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)

In [866]:
# Delete columns 'Var13' and 'application_date'
df.drop(['Var13'], axis=1, inplace=True)
df.drop(['application_date'], axis=1, inplace=True)

In [867]:
df.head()

,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,Var7,...,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28,working_months
0,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,108.73,...,5,NaN,2,0.0,Approved,1.0,NaN,0,0,62
1,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,195.99,...,2,NaN,4,0.0,Approved,NaN,1.0,0,0,160
2,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,248.34,...,5,5.0,1,0.0,Approved,NaN,NaN,0,0,248
3,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,3538.68,...,6,5.0,1,0.0,Approved,NaN,0.0,0,0,265
4,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,580.26,...,5,NaN,0,NaN,Rejected,1.0,NaN,0,0,248


Move 'working_months' to the first column

In [868]:
# Create a list of columns without 'working_months'
df_sorted_columns = df.columns.tolist()
df_sorted_columns.remove('working_months')
# Add 'working_months' to the first position
df_sorted_columns.insert(0, 'working_months')
# Reindex columns
df = df[df_sorted_columns]

In [869]:
# Update the list of date and numeric columns
date_columns = []
numeric_columns = numeric_columns + ['working_months']

In [870]:
df.head()

,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,62,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,160,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,248,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,265,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,248,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


### Handle binary variables

Convert the 'Application_status' column to binary variables to facilitate analysis. Applications marked as 'Approved' are changed to 1, and those marked as 'Rejected' are changed to 0. This conversion simplifies the analysis process, allowing for straightforward interpretation of application outcomes

In [871]:
# Check unique values for binary columns
for column in binary_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'target': [ 0. nan  1.]
Unique values for column 'Application_status': ['Approved' 'Rejected']
Unique values for column 'Var18': [ 1. nan  0.]
Unique values for column 'Var19': [nan  1.  0.]
Unique values for column 'Var27': [0 1]
Unique values for column 'Var28': [0 1]


As we can observe column "Application_status" needs to be modified

In [872]:
# Change value "Approved" to 1 and "Rejected" to 0 in column 'Application_status'
df['Application_status'] = df['Application_status'].replace({'Approved': 1, 'Rejected': 0})

In [873]:
# Check and count unique values for 'Application_status' to ensure that transformation was successful
df['Application_status'].value_counts(dropna=False)

1    36718
0    13282
Name: Application_status, dtype: int64

Check the rest of binary columns

In [874]:
df[binary_columns].isna().sum()

target                13282
Application_status        0
Var18                 37416
Var19                 29173
Var27                     0
Var28                     0
dtype: int64

In [875]:
df[binary_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   target              36718 non-null  float64
 1   Application_status  50000 non-null  int64  
 2   Var18               12584 non-null  float64
 3   Var19               20827 non-null  float64
 4   Var27               50000 non-null  int64  
 5   Var28               50000 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 2.3 MB


We can observe that we have some nulls in binary variables, but we will handle them later.

### Handle categorical variables

We have 5 categorical columns:

* Var2 - Loan purpose
* Var3 - Distribution channel
* Var11 - profession of main applicant
* Var12 - profession of second applicant
* Var14 - marital status of main applicant

All of them are treated as simple categorical variables(non-ordered)

In [876]:
# Check unique values for binary columns
for column in categorical_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'Var2': [ 2.  1.  3. nan]
Unique values for column 'Var3': ['1' '2' nan 'Direct' '3' 'Online']
Unique values for column 'Var11': [5 2 6 4 3 1 7]
Unique values for column 'Var12': [nan  5.  2.  4.  6.  7.  3.  1.]
Unique values for column 'Var14': [2 4 1 0 3]


According to the variables_description.xlsx file the variable Var2 should have the following values:

* 1 Direct
* 2 Broker
* 3 Online

In the dataset this variable has 5 unique values printed above. First of all we should transform 'Direct' to 1 and 'Online' to 3

In [877]:
# Check the number of observations in Var3 for each category
df['Var3'].value_counts(dropna=False)

1         24043
2         16130
3          8054
NaN        1403
Direct      325
Online       45
Name: Var3, dtype: int64

In [878]:
# Change the value "Direct" to 1 and "Online" to 3 in column "Var3"
df['Var3'] = df['Var3'].replace({'Direct': 1, 'Online': 3})

# Change the data type of column "Var3" to float (we still have NaN values)
df['Var3'] = df['Var3'].astype(float)

Check the unique values after transformation

In [879]:
# Check again the number of observations in Var3 for each category
df['Var3'].value_counts(dropna=False)

1.0    24368
2.0    16130
3.0     8099
NaN     1403
Name: Var3, dtype: int64

It looks that the transformation was successfull

In [880]:
# Count NA values for categorical columns
df[categorical_columns].isna().sum()

Var2      1403
Var3      1403
Var11        0
Var12    37538
Var14        0
dtype: int64

All categorical variables are ready for the further processing (filling NA values and one hot encoding).

### Handle NaN in target variable

As we can observe below we have 13282 NaN values in our target variable.

We found out that all of them are for "Application status" == rejected (1).

We analyze the approach to fill target variable of all rejected application to 1 (default). However this way would bring in a high conlusion that all of rejected people would default in the future and it would impact negatively on our model. Additionally we assume that we want to model the approved applications only.

After a long discussion we decided to delete all of rows with Application_status == 0.

In [881]:
# Check the number of NAa for target variable
df['target'].isna().sum()

13282

In [882]:
df.loc[df["Application_status"] == 0, "target"].isna().sum()

13282

In [883]:
# Drop rows where 'Application_status' == 0
df = df[df['Application_status'] != 0]

Now as we know that there are only rows with application_status == 0 we can delete this column from df

In [884]:
# Drop column 'Application_status'
df.drop(['Application_status'], axis=1, inplace=True)

In [885]:
# Delete "application_stauts" from binary_columns
binary_columns.remove('Application_status')

### Handle numeric data

In [886]:
df[numeric_columns].isna().sum()

ID                    0
customer_id           0
_r_                   0
Var9                  0
Var10             28043
Var1                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8              20538
Var15                 0
Var16                 0
Var17                32
Var20                 0
Var21                 0
Var22                 0
Var23                 0
Var24                 0
Var25              7401
Var26             14649
Var29                 0
Var30                 0
working_months        0
dtype: int64

#### Dependence of numeric variables with categorical variables

Var10 and Var8 are strictly related to Var1 and Var2 respectively, so we wanted to check the NAs in dependent variables.
However Var1 related to Var10 does not have any missings we moved to Var8 and Var2.

We wanted to check if there are any NA observations of Var2 (Loan purpose) having Var8 (Value of goods(car)) set to some value. If yes we assume that the value is entered correctly and we are missing Var2 for some unknown reason.

We decided to fill these rows with value 1 (meaning Car Loan).

As we can observe below we have 1018 NAs before filling the data.

In [887]:
# Count values inside Var8 group by value
df['Var2'].value_counts(dropna=False)

1.0    15744
3.0    10641
2.0     9315
NaN     1018
Name: Var2, dtype: int64

In [888]:
# Put value 1 to the column Var2 if Var8 is not NaN
df.loc[~df['Var8'].isna(), 'Var2'] = 1

After transformation the number of NaN in Var8 dropped by almost half

In [889]:
# count values inside Var8 group by value
df['Var2'].value_counts(dropna=False)

1.0    16180
3.0    10641
2.0     9315
NaN      582
Name: Var2, dtype: int64

#### Analyze Var9 and Var10

We have 28043 null values in Var10 (income of second applicant). We would like to check how many of these NAs is because of the fact that there is just one applicant.

As we can see below all of NAs are inside this group. We decided to sum these columns into one and name it "income".

In [890]:
# Count the number of nulls where Var1 = 1
df.loc[df["Var1"] == 1, "Var10"].isna().sum()

28043

In [891]:
# Create a new colum income
df["income"] = df["Var9"] + df["Var10"].fillna(0).astype(int) # Var9 was as integer, Var10 was as float, but we do not have NAs

In [892]:
# Drop columns Var9 and Var10
df.drop(["Var9", "Var10"], axis=1, inplace=True)

In [893]:
# update the list of numeric columns
numeric_columns.remove("Var9")
numeric_columns.remove("Var10")
numeric_columns.append("income")

Check again the number of null values

In [894]:
df[numeric_columns].isna().sum()

ID                    0
customer_id           0
_r_                   0
Var1                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8              20538
Var15                 0
Var16                 0
Var17                32
Var20                 0
Var21                 0
Var22                 0
Var23                 0
Var24                 0
Var25              7401
Var26             14649
Var29                 0
Var30                 0
working_months        0
income                0
dtype: int64

# Zmiana - kategoryczne ciągłe

Var8, Var25, Var26 - ciągła którą zamieniamy na kategoryczną + osobna kategoria na NA

[TODO]

We still have a few variables with nulls:

* Var8 - Value of the goods (car) 
* Var17 - Spendings estimation - the number of NAs is low, there is a sense in filling this, we decided to use KNN to fill these variables.
* Var25 - Amount on current account 
* Var26 - Amount on savings account

When it comes to Var25 and Var26 we decidec to fill NAa with 0. NAs means probably that the client does not have account in our bank and we cannot reach that value or does not have any money on either current or saving account. In both cases there is an element of uncertainty which we decided to minimize.

Because of this our model should be more conservative and penalize clients about whom we are not sure.

In [895]:
# Check the percentage number of NAs in Var8 related to whole df
round(df['Var8'].isna().sum() / df.shape[0] * 100, 2)

55.93

In [896]:
df[numeric_columns].isna().sum()

ID                    0
customer_id           0
_r_                   0
Var1                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8              20538
Var15                 0
Var16                 0
Var17                32
Var20                 0
Var21                 0
Var22                 0
Var23                 0
Var24                 0
Var25              7401
Var26             14649
Var29                 0
Var30                 0
working_months        0
income                0
dtype: int64

### Handle remaining NULL values

In [897]:
df.isnull().sum()

working_months        0
ID                    0
customer_id           0
_r_                   0
Var1                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8              20538
Var15                 0
Var16                 0
Var17                32
Var20                 0
Var21                 0
Var22                 0
Var23                 0
Var24                 0
Var25              7401
Var26             14649
Var29                 0
Var30                 0
Var2                582
Var3               1018
Var11                 0
Var12             28043
Var14                 0
target                0
Var18             27125
Var19             20538
Var27                 0
Var28                 0
income                0
dtype: int64

* Var17 - spedning estimation - as we wrote before, we will fill with KNN
* Var2 - loan purpose [categorical] - drop NA rows
* Var3 - Distribution channel [categorical] - drop NA rows
* Var12 - profession of second applicant [categorical] - add new category
* Var18 - Property ownership for property renovation
* Var19 - Clasification of the vehicle (Car, Motorbike)

When it comes to the Var18 and Var19 we analyze combining them with Var2 (loan purpose) and have 1 categorical variable with values:

1. Car loan motorbike
2. Car loan car
3. House renovation your property
4. House renovation not your property
5. Short cash

We finally decided not to use this approach, however we provide you with the short analysis why.

Var18, Var19 - dodać kolejną kategorię

#### Var12

Check if all NAs in Var12 (profession of second applicant)are if Var1 == 1, so if there is just one applicant.

The number of NAs in this category is the same as the number of NAs when Var1 == 1, so we fill it with a new category.

In [898]:
df.loc[df["Var1"] == 1, "Var12"].isna().sum()

28043

Fill NA in Var12 (profession of second applicant) with new category: 8 - just one applicant

In [899]:
df["Var12"] = df["Var12"].fillna(8)

#### Var18 and Var19 - check

Check if we can combine Var2 with Var18 and Var19 and having one categorical variable instead of one vategorical and 2 binary with huge amount of nulls. The new variable would consists of 5 values:

1. Car loan motorbike
2. Car loan car
3. House renovation your property
4. House renovation not your property
5. Short cash

First of all we need to check the values - if combining them would reduce the nulls to 0 and nulls are related only to the variable specificity or we really have some missing data.

Var2 == 1 is related to Var 19 and Var2 == 2 is related to Var 18.

In [900]:
df.shape[0]

36718

In [901]:
# Check if the number of "conditional" values is the same as the number of rows in df

# Count rows where Var2 == 1 and Var19 is not NaN
var2_19 = df[(df['Var2'] == 1) & (~df['Var19'].isna())].shape[0]

# Count rows where Var2 == 2 and Var 18 is not NaN
var2_18 = df[(df['Var2'] == 2) & (~df['Var18'].isna())].shape[0]

# Count rows where Var2 == 3
var2_3 = df.loc[df["Var2"] == 3].shape[0]

# Calculate of sum
sum = var2_19 + var2_18 + var2_3

print(var2_18)
print(var2_19)
print(var2_3)
print(sum)

9315
16180
10641
36136


The number of rows in df is 36718 while the "conditional" sum is 36136, meaning that there would be a new category or we would need to add original values from Var2 and the number of categories would increase.

Lets try to create this categorical column and see the distribution.

In [902]:
# Add new column loan_desc
df['loan_desc'] = 0

# Check conditions
df.loc[(df['Var2'] == 1) & (df['Var19'] == 1), 'loan_desc'] = 1
df.loc[(df['Var2'] == 1) & (df['Var19'] == 0), 'loan_desc'] = 2
df.loc[(df['Var2'] == 2) & (df['Var18'] == 1), 'loan_desc'] = 3
df.loc[(df['Var2'] == 2) & (df['Var18'] == 0), 'loan_desc'] = 4
df.loc[df['Var2'] == 3, 'loan_desc'] = 5

In [903]:
df['loan_desc'].value_counts(dropna=False)

1    11823
5    10641
3     7840
2     4357
4     1475
0      582
Name: loan_desc, dtype: int64

As we can observe above introducing new variables diveded the df into smaller, not very numerous groups. Vast majority of data are concentrated around 1, 5 and 3 group so probably (there was no precise description of variable Var18 and Var19) for:

 * Car loan for car
 * House renovation for client's property
 * Short cash

These new two groups (2, 4) and additional 0 constitute a small percentage of all df.

Summarizing we decide to go with variable Var2 and delete Var18 and Var19.

In [904]:
df.shape

(36718, 34)

#### Drop Var18 i Var19

In [905]:
# Drop Var18 and Var19 columns
df.drop('Var18', axis=1, inplace=True)
df.drop('Var19', axis=1, inplace=True)

In [906]:
# Update the list of binary columns
binary_columns.remove('Var18')
binary_columns.remove('Var19')

#### Var2 and Var3 - drop NAs

In [907]:
# Drop NA rows fof Var2
df = df[~df['Var2'].isna()]
df = df[~df['Var3'].isna()]

In [908]:
# sum number of observations for each category in Var3
df['Var3'].value_counts(dropna=False)

1.0    18354
2.0    12066
3.0     5280
Name: Var3, dtype: int64

In [909]:
df.isna().sum()

working_months        0
ID                    0
customer_id           0
_r_                   0
Var1                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8              19956
Var15                 0
Var16                 0
Var17                31
Var20                 0
Var21                 0
Var22                 0
Var23                 0
Var24                 0
Var25              7172
Var26             14201
Var29                 0
Var30                 0
Var2                  0
Var3                  0
Var11                 0
Var12                 0
Var14                 0
target                0
Var27                 0
Var28                 0
income                0
loan_desc             0
dtype: int64

Summarizing we still have 31 NAs in Var17 which will be filled using KNN

### Handling numeric/categorical NAs

Var8, Var25, Var26 - ciągła którą zamieniamy na kategoryczną + osobna kategoria na NA

#### Var8

In [910]:
def quantile_cut_and_merge(df, column_name):
    # Filter out NaN values from the specified column
    df_filtered = df.dropna(subset=[column_name])
    
    # Perform quantile cut on filtered DataFrame
    df_filtered[column_name + '_q'] = pd.qcut(df_filtered[column_name], 10, labels=False, duplicates='drop')
    
    # Merge the results back into the original DataFrame
    df = df.merge(df_filtered[['ID', column_name + '_q']], how='left', on='ID')
    
    # Fill NaN values in the new quantile column with 10
    df[column_name + '_q'] = df[column_name + '_q'].fillna(10)
    
    # Drop the original column
    df.drop(column_name, axis=1, inplace=True)
    
    return df

In [911]:
for col in ['Var8', 'Var25', 'Var26']:
    df = quantile_cut_and_merge(df, col)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_56015/3854490666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[column_name + '_q'] = pd.qcut(df_filtered[column_name], 10, labels=False, duplicates='drop')
/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_56015/3854490666.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[column_name + '_q'] = pd.qcut(df_filtered[column_name], 10, labels=False, duplicates='drop')
/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn

#### Handle categorical variables without NAs

In [912]:
# Continuous columns to be changed to categorical
columns = ['working_months', 'Var4', 'Var5', 'Var7', 'income', 'Var24', 'Var29', 'Var30']

for col in columns:
    # Perform quantile cut on filtered DataFrame
    df[col] = pd.qcut(df[col], 10, labels=False, duplicates='drop')

#### _r_ - drop

In [913]:
# Drop _r_ column
df.drop('_r_', axis=1, inplace=True)

In [914]:
df

,working_months,ID,customer_id,Var1,Var4,Var5,Var6,Var7,Var15,Var16,...,Var12,Var14,target,Var27,Var28,income,loan_desc,Var8_q,Var25_q,Var26_q
0,2,11034977,32537148,1,2,9,1,0,0,0,...,8.0,2,0.0,0,0,1,3,10.0,3.0,0.0
1,5,11034978,32761663,1,3,9,1,0,4,4,...,8.0,4,0.0,0,0,0,1,0.0,3.0,1.0
2,9,11034979,32701063,2,0,2,1,0,0,0,...,5.0,1,0.0,0,0,5,5,10.0,3.0,0.0
3,9,11034980,32386786,3,3,5,6,9,0,0,...,5.0,1,0.0,0,0,2,2,0.0,1.0,4.0
4,0,11034982,32861919,1,2,6,1,0,1,1,...,8.0,0,0.0,0,0,0,4,10.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35695,7,11084969,32845249,1,1,0,6,9,0,0,...,8.0,0,0.0,0,0,8,5,10.0,10.0,10.0
35696,8,11084970,32623752,1,7,2,3,9,1,1,...,8.0,1,0.0,0,0,8,3,10.0,5.0,9.0
35697,5,11084972,32554904,2,9,9,1,2,0,1,...,5.0,0,1.0,0,0,8,2,8.0,10.0,10.0
35698,6,11084974,32433438,1,9,2,1,9,0,0,...,8.0,0,0.0,0,0,9,1,9.0,5.0,9.0


#### Zmienne problematyczne:

* Var20
* Var21
* Var22
* Var30

#### Var20

In [915]:
df['Var20'].value_counts(dropna=False)

0     23044
1      8430
2      2699
3       887
4       358
5       143
6        70
7        34
8        16
9        10
11        3
10        3
13        2
15        1
Name: Var20, dtype: int64

In [916]:
# create a new column Var20_1 filled basing on Var20
def process_var(var, df):
    df[var + '_1'] = 0
    df.loc[df[var].isin([1, 2]), var + '_1'] = 1
    df.loc[df[var].isin([3, 4, 5]), var + '_1'] = 2
    df.loc[df[var] > 5, var + '_1'] = 3
    return df

In [917]:
df = process_var('Var20', df)

In [918]:
df['Var20_1'].value_counts(dropna=False)

0    23044
1    11129
2     1388
3      139
Name: Var20_1, dtype: int64

In [919]:
df.drop('Var20', axis=1, inplace=True)

#### Var 21

In [920]:
df['Var21'].value_counts(dropna=False)

0     16659
1     10280
2      4856
3      2099
4       931
5       449
6       212
7        91
8        47
9        39
10       15
11        6
12        6
13        4
14        3
16        1
18        1
15        1
Name: Var21, dtype: int64

In [921]:
df = process_var('Var21', df)

In [922]:
df['Var21_1'].value_counts(dropna=False)

0    16659
1    15136
2     3479
3      426
Name: Var21_1, dtype: int64

In [923]:
df.drop('Var21', axis=1, inplace=True)

#### Var22

In [924]:
df['Var22'].value_counts(dropna=False)

0     12936
1     10490
2      5960
3      3098
4      1586
5       781
6       406
7       190
8       101
9        66
10       31
11       21
12       14
13        8
14        6
16        3
15        2
18        1
Name: Var22, dtype: int64

In [925]:
df = process_var('Var22', df)

In [926]:
df['Var22_1'].value_counts(dropna=False)

1    16450
0    12936
2     5465
3      849
Name: Var22_1, dtype: int64

In [927]:
df.drop('Var22', axis=1, inplace=True)

#### Var30

Leave as is

In [928]:
df['Var30'].value_counts(dropna=False)

6    7834
1    5910
0    4081
7    4020
4    3616
5    3508
3    3421
2    3310
Name: Var30, dtype: int64

In [929]:
df.isna().sum()

working_months     0
ID                 0
customer_id        0
Var1               0
Var4               0
Var5               0
Var6               0
Var7               0
Var15              0
Var16              0
Var17             31
Var23              0
Var24              0
Var29              0
Var30              0
Var2               0
Var3               0
Var11              0
Var12              0
Var14              0
target             0
Var27              0
Var28              0
income             0
loan_desc          0
Var8_q             0
Var25_q            0
Var26_q            0
Var20_1            0
Var21_1            0
Var22_1            0
dtype: int64

### Filling NAs with KNN

In [930]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35700 entries, 0 to 35699
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  35700 non-null  int64  
 1   ID              35700 non-null  int64  
 2   customer_id     35700 non-null  int64  
 3   Var1            35700 non-null  int64  
 4   Var4            35700 non-null  int64  
 5   Var5            35700 non-null  int64  
 6   Var6            35700 non-null  int64  
 7   Var7            35700 non-null  int64  
 8   Var15           35700 non-null  int64  
 9   Var16           35700 non-null  int64  
 10  Var17           35669 non-null  float64
 11  Var23           35700 non-null  int64  
 12  Var24           35700 non-null  int64  
 13  Var29           35700 non-null  int64  
 14  Var30           35700 non-null  int64  
 15  Var2            35700 non-null  float64
 16  Var3            35700 non-null  float64
 17  Var11           35700 non-null 

In [931]:
# change working_months to int32
df['working_months'] = df['working_months'].astype('int32')

In [932]:
# Set the seed for reproducibility
np.random.seed(123456)

# Save the original data types of the DataFrame. This is crucial for restoring the DataFrame's data types
# after imputation since KNNImputer will convert the imputed data to float64 by default.
original_dtypes = df.dtypes

# Initialize the KNNImputer. The default setting uses 5 nearest neighbors to impute missing values,
# which is a balance between accuracy and computational efficiency.
knn_imputer = KNNImputer()

# Replace 0s with NaNs in the 'working_months' column if necessary. This step is preparatory for imputation,
# ensuring that 0s, which may signify missing or unrecorded data, are correctly treated as missing values.
df['working_months'] = df['working_months'].replace(0, np.nan)

# Select only numerical columns from the DataFrame for imputation. KNNImputer works with numerical data,
# so identifying these columns is a prerequisite for the imputation process.
numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32']).columns

# Perform the imputation using KNN. This process fills in missing values based on the mean of the
# nearest neighbors found in the multidimensional space, providing a reasonable estimate for missing data.
imputed_data = knn_imputer.fit_transform(df[numerical_cols])

# Convert the imputed array back into a DataFrame, preserving the original index. This step ensures that
# the imputed data retains the structure and index of the original DataFrame for consistency and ease of analysis.
imputed_df = pd.DataFrame(imputed_data, columns=numerical_cols, index=df.index)

# Restore the original data types. After imputation, it's important to convert the data back to its
# original types for accurate analysis and consistency with the rest of the dataset.
for col in numerical_cols:
    if original_dtypes[col].kind in 'iu':  # Check if the data type is integer
        imputed_df[col] = imputed_df[col].round().astype(original_dtypes[col])
    else:  # For floating-point types, retain as is
        imputed_df[col] = imputed_df[col].astype(original_dtypes[col])

# Update the original 'sorted_data_encoded' DataFrame with the imputed data. This final step integrates
# the imputed numerical data back into the main DataFrame, completing the imputation process.
df.update(imputed_df)

# Display the DataFrame information to check the results of the imputation and data type restoration.
# This overview allows for a quick verification that the dataset is now complete with imputed values
# and that all columns are of their appropriate data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35700 entries, 0 to 35699
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  35700 non-null  float64
 1   ID              35700 non-null  int64  
 2   customer_id     35700 non-null  int64  
 3   Var1            35700 non-null  int64  
 4   Var4            35700 non-null  int64  
 5   Var5            35700 non-null  int64  
 6   Var6            35700 non-null  int64  
 7   Var7            35700 non-null  int64  
 8   Var15           35700 non-null  int64  
 9   Var16           35700 non-null  int64  
 10  Var17           35700 non-null  float64
 11  Var23           35700 non-null  int64  
 12  Var24           35700 non-null  int64  
 13  Var29           35700 non-null  int64  
 14  Var30           35700 non-null  int64  
 15  Var2            35700 non-null  float64
 16  Var3            35700 non-null  float64
 17  Var11           35700 non-null 

### Change Var 17 to categorical

In [933]:
df

,working_months,ID,customer_id,Var1,Var4,Var5,Var6,Var7,Var15,Var16,...,Var27,Var28,income,loan_desc,Var8_q,Var25_q,Var26_q,Var20_1,Var21_1,Var22_1
0,2.0,11034977,32537148,1,2,9,1,0,0,0,...,0,0,1,3,10.0,3.0,0.0,1,1,1
1,5.0,11034978,32761663,1,3,9,1,0,4,4,...,0,0,0,1,0.0,3.0,1.0,0,0,0
2,9.0,11034979,32701063,2,0,2,1,0,0,0,...,0,0,5,5,10.0,3.0,0.0,1,1,1
3,9.0,11034980,32386786,3,3,5,6,9,0,0,...,0,0,2,2,0.0,1.0,4.0,0,0,0
4,6.0,11034982,32861919,1,2,6,1,0,1,1,...,0,0,0,4,10.0,0.0,10.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35695,7.0,11084969,32845249,1,1,0,6,9,0,0,...,0,0,8,5,10.0,10.0,10.0,1,2,2
35696,8.0,11084970,32623752,1,7,2,3,9,1,1,...,0,0,8,3,10.0,5.0,9.0,0,0,0
35697,5.0,11084972,32554904,2,9,9,1,2,0,1,...,0,0,8,2,8.0,10.0,10.0,1,1,1
35698,6.0,11084974,32433438,1,9,2,1,9,0,0,...,0,0,9,1,9.0,5.0,9.0,0,0,0


In [934]:
col = 'Var17'
df['Var17_1'] = pd.qcut(df[col], 10, labels=False, duplicates='drop')

In [935]:
df['Var17_1'].value_counts(dropna=False)

3    3570
0    3570
5    3570
1    3570
2    3570
4    3570
6    3570
7    3570
9    3570
8    3570
Name: Var17_1, dtype: int64

In [936]:
df.drop('Var17', axis=1, inplace=True)

### Final check of df

In [937]:
df.head()

,working_months,ID,customer_id,Var1,Var4,Var5,Var6,Var7,Var15,Var16,...,Var28,income,loan_desc,Var8_q,Var25_q,Var26_q,Var20_1,Var21_1,Var22_1,Var17_1
0,2.0,11034977,32537148,1,2,9,1,0,0,0,...,0,1,3,10.0,3.0,0.0,1,1,1,3
1,5.0,11034978,32761663,1,3,9,1,0,4,4,...,0,0,1,0.0,3.0,1.0,0,0,0,0
2,9.0,11034979,32701063,2,0,2,1,0,0,0,...,0,5,5,10.0,3.0,0.0,1,1,1,5
3,9.0,11034980,32386786,3,3,5,6,9,0,0,...,0,2,2,0.0,1.0,4.0,0,0,0,1
4,6.0,11034982,32861919,1,2,6,1,0,1,1,...,0,0,4,10.0,0.0,10.0,0,1,1,0


In [938]:
df.isna().sum()

working_months    0
ID                0
customer_id       0
Var1              0
Var4              0
Var5              0
Var6              0
Var7              0
Var15             0
Var16             0
Var23             0
Var24             0
Var29             0
Var30             0
Var2              0
Var3              0
Var11             0
Var12             0
Var14             0
target            0
Var27             0
Var28             0
income            0
loan_desc         0
Var8_q            0
Var25_q           0
Var26_q           0
Var20_1           0
Var21_1           0
Var22_1           0
Var17_1           0
dtype: int64

In [939]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35700 entries, 0 to 35699
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  35700 non-null  float64
 1   ID              35700 non-null  int64  
 2   customer_id     35700 non-null  int64  
 3   Var1            35700 non-null  int64  
 4   Var4            35700 non-null  int64  
 5   Var5            35700 non-null  int64  
 6   Var6            35700 non-null  int64  
 7   Var7            35700 non-null  int64  
 8   Var15           35700 non-null  int64  
 9   Var16           35700 non-null  int64  
 10  Var23           35700 non-null  int64  
 11  Var24           35700 non-null  int64  
 12  Var29           35700 non-null  int64  
 13  Var30           35700 non-null  int64  
 14  Var2            35700 non-null  float64
 15  Var3            35700 non-null  float64
 16  Var11           35700 non-null  int64  
 17  Var12           35700 non-null 

#### Write df as csv

In [940]:
df.to_csv('../data/output/development_sample_cleaned_categorical.csv', index = False)  # Save the cleaned data to a new CSV file